## Ref 

- https://cloud.google.com/vision/
- https://cloud.google.com/vision/docs/how-to
- https://pypi.org/project/google-cloud-vision/

In [1]:
# python 3 



# gcloud 
from google.cloud import vision
from google.oauth2 import service_account

# OP 
import pandas as pd 
import numpy as np 

#--------------------------------------------------
# config 
# save ur google cloud credentials below
credentials = service_account.Credentials.from_service_account_file('/Users/yennanliu/Downloads/google_cloud_creds2.json')
client = vision.ImageAnnotatorClient(credentials=credentials)


In [3]:


#--------------------------------------------------
# help func 

def call_google_image_api(url):
    response = client.annotate_image({
    'image': {'source': {'image_uri': url}},
    'features': [{'type': vision.enums.Feature.Type.FACE_DETECTION}],})
    print ('response : ', response)
    return response




def call_google_handwritten_api(uri):

    """
    # ref 
    https://cloud.google.com/vision/docs/ocr

    # put ur image url above 
    # e.g. https://s-i.huffpost.com/gen/2165258/images/n-NY-TIMES-ERROR-628x314.jpg

    """

    """Detects handwritten characters in the file located in Google Cloud
    Storage.

    Args:
    uri: The path to the file in Google Cloud Storage (gs://...)
    """
    from google.cloud import vision_v1p3beta1 as vision
    credentials = service_account.Credentials.from_service_account_file('google_cloud_creds2.json')
    
    
    client = vision.ImageAnnotatorClient(credentials=credentials)
    image = vision.types.Image()
    image.source.image_uri = uri

    # Language hint codes for handwritten OCR:
    # en-t-i0-handwrit, mul-Latn-t-i0-handwrit
    # Note: Use only one language hint code per request for handwritten OCR.
    image_context = vision.types.ImageContext(
        language_hints=['en-t-i0-handwrit'])

    response = client.document_text_detection(image=image,
                                              image_context=image_context)

    print('Full Text: {}'.format(response.full_text_annotation.text))
    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            print('\nBlock confidence: {}\n'.format(block.confidence))

            for paragraph in block.paragraphs:
                print('Paragraph confidence: {}'.format(
                    paragraph.confidence))

                for word in paragraph.words:
                    word_text = ''.join([
                        symbol.text for symbol in word.symbols
                    ])
                    print('Word text: {} (confidence: {})'.format(
                        word_text, word.confidence))

                    for symbol in word.symbols:
                        print('\tSymbol: {} (confidence: {})'.format(
                            symbol.text, symbol.confidence))


#--------------------------------------------------


In [8]:
uri = 'https://cbsnews1.cbsistatic.com/hub/i/2018/09/24/bbee124f-bbe8-4d73-80c3-7e681214e062/0924-cbsn-trumpkimsummit-ayfayg-1665758-640x360.jpg'

print ('uri :' , uri)
response = call_google_image_api(uri)

uri : https://cbsnews1.cbsistatic.com/hub/i/2018/09/24/bbee124f-bbe8-4d73-80c3-7e681214e062/0924-cbsn-trumpkimsummit-ayfayg-1665758-640x360.jpg
response :  face_annotations {
  bounding_poly {
    vertices {
      x: 277
      y: 34
    }
    vertices {
      x: 425
      y: 34
    }
    vertices {
      x: 425
      y: 206
    }
    vertices {
      x: 277
      y: 206
    }
  }
  fd_bounding_poly {
    vertices {
      x: 288
      y: 73
    }
    vertices {
      x: 387
      y: 73
    }
    vertices {
      x: 387
      y: 172
    }
    vertices {
      x: 288
      y: 172
    }
  }
  landmarks {
    type: LEFT_EYE
    position {
      x: 312.9682922363281
      y: 105.73651123046875
      z: 0.00100697239395231
    }
  }
  landmarks {
    type: RIGHT_EYE
    position {
      x: 347.94708251953125
      y: 104.90869903564453
      z: -23.364290237426758
    }
  }
  landmarks {
    type: LEFT_OF_LEFT_EYEBROW
    position {
      x: 301.98956298828125
      y: 96.25361633300781
     

In [9]:
type(response)

google.cloud.vision_v1.types.AnnotateImageResponse

In [15]:
# transform google cloud vision api response to dict 

from google.protobuf.json_format import MessageToDict
dict_obj = MessageToDict(response)


In [16]:
dict_obj

{'faceAnnotations': [{'angerLikelihood': 'UNLIKELY',
   'blurredLikelihood': 'VERY_UNLIKELY',
   'boundingPoly': {'vertices': [{'x': 277, 'y': 34},
     {'x': 425, 'y': 34},
     {'x': 425, 'y': 206},
     {'x': 277, 'y': 206}]},
   'detectionConfidence': 0.9652358293533325,
   'fdBoundingPoly': {'vertices': [{'x': 288, 'y': 73},
     {'x': 387, 'y': 73},
     {'x': 387, 'y': 172},
     {'x': 288, 'y': 172}]},
   'headwearLikelihood': 'VERY_UNLIKELY',
   'joyLikelihood': 'VERY_UNLIKELY',
   'landmarkingConfidence': 0.43969130516052246,
   'landmarks': [{'position': {'x': 312.9682922363281,
      'y': 105.73651123046875,
      'z': 0.00100697239395231},
     'type': 'LEFT_EYE'},
    {'position': {'x': 347.94708251953125,
      'y': 104.90869903564453,
      'z': -23.364290237426758},
     'type': 'RIGHT_EYE'},
    {'position': {'x': 301.98956298828125,
      'y': 96.25361633300781,
      'z': 13.67840576171875},
     'type': 'LEFT_OF_LEFT_EYEBROW'},
    {'position': {'x': 317.3613586425